# Loading Responses

In [1]:
options(warn=1)
if(!require('lme4')) {
    install.packages('lme4', repos='https://cloud.r-project.org')
    library('lme4')
}
if(!require('readr')) {
    install.packages('readr', repos='https://cloud.r-project.org')
    library('readr')
}
if(!require('marginaleffects')) {
    install.packages('marginaleffects', repos='https://cloud.r-project.org')
    library('marginaleffects')
}
if(!require('emmeans')) {
    install.packages('emmeans', repos='https://cloud.r-project.org')
    library('emmeans')
}
if(!require('car')) {
    install.packages('car', repos='https://cloud.r-project.org')
    library('car')
}
df <- read_csv("result_1743457603_20250506_20250506F.csv", na = "empty", col_select = c("Accuracy", "Group", "GroupKind", "Angle", "Proximity", "n_candidates", "Actor", "Candidates", "Stimulus_ID", "Prompt_ID", "Participant_ID", "list_id", "Run_ID", "Part"), col_types = cols(
    Accuracy = col_logical(),
    Group = col_factor(),
    GroupKind = col_factor(),
    Angle = col_factor(),
    Proximity = col_integer(),
    n_candidates = col_integer(),
    Actor = col_factor(c('X', 'Y')),
    Candidates = col_factor(),
    Stimulus_ID = col_factor(),
    Prompt_ID = col_factor(),
    Participant_ID = col_factor(),
    list_id = col_factor(),
    Run_ID = col_character(),
    Part = col_character(),
),show_col_types = TRUE)
df <- df[df$Group == "gpt-4o" & df$list_id != -1 & df$Part != "p0", ]
df$offset <- log(1/df$n_candidates / (1 - 1/df$n_candidates))

Loading required package: lme4

Loading required package: Matrix

Loading required package: readr

Loading required package: marginaleffects

Loading required package: emmeans

Welcome to emmeans.
Caution: You lose important information if you filter this package's results.
See '? untidy'

Loading required package: car

Loading required package: carData

Rows: 156825 Columns: 14
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (2): Run_ID, Part
int (2): Proximity, n_candidates
lgl (1): Accuracy
fct (9): Stimulus_ID, Prompt_ID, Participant_ID, Group, GroupKind, Angle, Ac...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [2]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: aarch64-apple-darwin20.0.0 (64-bit)
Running under: macOS 15.5

Matrix products: default
BLAS/LAPACK: /Users/zory/miniforge3/envs/py310/lib/libopenblas.0.dylib;  LAPACK version 3.12.0

locale:
[1] C

time zone: America/Los_Angeles
tzcode source: system (macOS)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] car_3.1-3              carData_3.0-5          emmeans_1.11.0        
[4] marginaleffects_0.25.1 readr_2.1.5            lme4_1.1-35.5         
[7] Matrix_1.6-5          

loaded via a namespace (and not attached):
 [1] bit_4.6.0          jsonlite_2.0.0     compiler_4.3.3     crayon_1.5.3      
 [5] tidyselect_1.2.1   Rcpp_1.0.14        IRdisplay_1.1      parallel_4.3.3    
 [9] splines_4.3.3      boot_1.3-31        uuid_1.2-1         fastmap_1.2.0     
[13] IRkernel_1.3.2     lattice_0.22-6     R6_2.6.1           generics_0.1.4    
[17] Formula_1.2-5      MASS_7.3-60

# Fitting models

In [3]:
model0 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Candidates) + (1|Stimulus_ID) + (1|Prompt_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

boundary (singular) fit: see help('isSingular')



In [4]:
model1 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Stimulus_ID) + (1|Prompt_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model1

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Stimulus_ID) + (1 | Prompt_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
 7939.456  7996.296 -3961.728  7923.456      8992 
Random effects:
 Groups      Name        Std.Dev.
 Stimulus_ID (Intercept) 3.06101 
 Prompt_ID   (Intercept) 0.08895 
Number of obs: 9000, groups:  Stimulus_ID, 900; Prompt_ID, 12
Fixed Effects:
                       (Intercept)                           Angleleft  
                            0.8451                             -0.1712  
                        Angleright     scale(Proximity, scale = FALSE)  
                           -0.1276                             -0.4647  
scale(n_candidates, scale = FALSE)                              ActorY  
                           -0.5869   

In [5]:
model2 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Candidates) + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model2

boundary (singular) fit: see help('isSingular')



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Actor:Candidates) + (1 | Stimulus_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
 7940.747  7997.587 -3962.374  7924.747      8992 
Random effects:
 Groups           Name        Std.Dev. 
 Stimulus_ID      (Intercept) 3.058e+00
 Actor:Candidates (Intercept) 8.831e-05
Number of obs: 9000, groups:  Stimulus_ID, 900; Actor:Candidates, 18
Fixed Effects:
                       (Intercept)                           Angleleft  
                            0.8427                             -0.1698  
                        Angleright     scale(Proximity, scale = FALSE)  
                           -0.1262                             -0.4639  
scale(n_candidates, scale = FALSE)                              ActorY  
     

In [6]:
model3 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

In [7]:
model3_5 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Candidates), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

# Model selection

In [8]:
anova(model1, model3)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model3,7,7938.748,7988.483,-3962.374,7924.748,NA,NA,NA
model1,8,7939.456,7996.296,-3961.728,7923.456,1.291672,1,0.25574


In [9]:
anova(model3, model3_5)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
model3,7,7938.748,7988.483,-3962.374,7924.748,NA,NA,NA
model3_5,7,11911.919,11961.654,-5948.959,11897.919,0,0,NA


In [10]:
model <- model3

# Interpretation

In [11]:
summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Stimulus_ID)
   Data: df
 Offset: offset

     AIC      BIC   logLik deviance df.resid 
  7938.7   7988.5  -3962.4   7924.7     8993 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.9170 -0.3529  0.1520  0.3589  3.0744 

Random effects:
 Groups      Name        Variance Std.Dev.
 Stimulus_ID (Intercept) 9.35     3.058   
Number of obs: 9000, groups:  Stimulus_ID, 900

Fixed effects:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                          0.8428     0.2180   3.865 0.000111 ***
Angleleft                           -0.1698     0.2797  -0.607 0.543743    
Angleright                          -0.1262     0.2784  -0.453 0.650207    
scale(Proximity, scale = FALSE)     -0.4639     0.14

In [12]:
vc_list <- lapply(VarCorr(model), function(x) attr(x, "stddev")^2)
vc <- sum(unlist(vc_list))
effect_size_denom_t <- sqrt(vc + pi^2/3)

res <- summary(model)$coefficients
res <- as.data.frame(res)
res <- cbind(term = rownames(res), res)
rownames(res) <- NULL
res$`Pr(>|z|)` <- format(res$`Pr(>|z|)`, scientific = FALSE)
res$CI_low <- res$Estimate - 1.96 * res$`Std. Error`
res$CI_high <- res$Estimate + 1.96 * res$`Std. Error`
res$effect_size_denom <- effect_size_denom_t
res$effect_size <- res$Estimate / effect_size_denom_t
res

term,Estimate,Std. Error,z value,Pr(>|z|),CI_low,CI_high,effect_size_denom,effect_size
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.8427575,0.2180374,3.8651962,0.00011099994,0.4154041,1.2701109,3.55532,0.23704122
Angleleft,-0.1698470,0.2797391,-0.6071621,0.54374332174,-0.7181356,0.3784416,3.55532,-0.04777262
Angleright,-0.1262396,0.2783831,-0.4534744,0.65020717599,-0.6718704,0.4193912,3.55532,-0.03550723
"scale(Proximity, scale = FALSE)",-0.4639305,0.1401113,-3.3111582,0.00092910658,-0.7385486,-0.1893125,3.55532,-0.13048909
"scale(n_candidates, scale = FALSE)",-0.5870535,0.1527254,-3.8438498,0.00012111914,-0.8863953,-0.2877117,3.55532,-0.16511972
ActorY,-0.9381818,0.2329477,-4.0274351,0.00005638863,-1.3947594,-0.4816043,3.55532,-0.26388109


In [13]:
avg_predictions(model, by="Angle", hypothesis = ~ pairwise)

hypothesis,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(left) - (front),-0.029908795,0.03275723,-0.91304415,0.3612193,1.46905303,-0.09411178,0.03429419
(right) - (front),-0.032086375,0.03482471,-0.92136817,0.3568582,1.48657701,-0.10034154,0.03616879
(right) - (left),-0.002177581,0.03089153,-0.07049119,0.9438027,0.08344278,-0.06272387,0.05836871


In [14]:
avg_predictions(model, by="Proximity")

Proximity,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.5931320,0.02120830,27.96698,4.098444e-172,569.3366,0.5515645,0.6346995
2,0.5262255,0.01292145,40.72494,0.000000e+00,Inf,0.5008999,0.5515511
3,0.4896932,0.02185856,22.40281,3.695225e-111,366.8484,0.4468512,0.5325352


In [15]:
avg_slopes(model, variables ="Proximity", type = "link")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Proximity,dY/dX,-0.4639305,0.1401064,-3.311272,0.000928728,10.07246,-0.7385341,-0.189327,-0.3571926,-0.3572854,-0.357239


In [16]:
avg_predictions(model, by="n_candidates")

n_candidates,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,0.5789300,0.01765427,32.79262,7.501477e-236,781.0679,0.5443282,0.6135317
3,0.5119723,0.01636753,31.27976,8.796131e-215,711.0777,0.4798925,0.5440520
4,0.4211505,0.02304367,18.27619,1.280681e-74,245.4658,0.3759857,0.4663152


In [17]:
avg_slopes(model, variables ="n_candidates", type = "link")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
n_candidates,dY/dX,-0.5870535,0.1527299,-3.843737,0.0001211747,13.01062,-0.8863986,-0.2877085,-0.3571803,-0.3572977,-0.357239


In [18]:
avg_predictions(model, by="Actor", hypothesis = ~ pairwise)

hypothesis,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Y) - (X),-0.1191365,0.02731737,-4.361198,1.293521e-05,16.23834,-0.1726775,-0.06559541


The below code is for generating the table in the paper.

In [19]:
stim_id_var_val <- if (!is.null(vc_list$Stimulus_ID) && length(vc_list$Stimulus_ID) > 0) vc_list$Stimulus_ID[[1]] else NA_real_
prompt_id_var_val <- if (!is.null(vc_list$Prompt_ID) && length(vc_list$Prompt_ID) > 0) vc_list$Prompt_ID[[1]] else NA_real_
participant_id_var_val <- if (!is.null(vc_list$Participant_ID) && length(vc_list$Participant_ID) > 0) vc_list$Participant_ID[[1]] else NA_real_

intercept_est <- res[res$term == "(Intercept)", "Estimate"]
intercept_se <- res[res$term == "(Intercept)", "Std. Error"]

angleleft_est <- res[res$term == "Angleleft", "Estimate"]
angleleft_se <- res[res$term == "Angleleft", "Std. Error"]
angleleft_p_val <- res[res$term == "Angleleft", "Pr(>|z|)"]
angleleft_eff_size <- res[res$term == "Angleleft", "effect_size"]

angleright_est <- res[res$term == "Angleright", "Estimate"]
angleright_se <- res[res$term == "Angleright", "Std. Error"]
angleright_p_val <- res[res$term == "Angleright", "Pr(>|z|)"]
angleright_eff_size <- res[res$term == "Angleright", "effect_size"]

proximity_term_name <- "scale(Proximity, scale = FALSE)"
if (!proximity_term_name %in% res$term) {
    proximity_term_name <- grep("Proximity", res$term, value = TRUE)[1]
}
proximity_est <- res[res$term == proximity_term_name, "Estimate"]
proximity_se <- res[res$term == proximity_term_name, "Std. Error"]
proximity_p_val <- res[res$term == proximity_term_name, "Pr(>|z|)"]
proximity_eff_size <- res[res$term == proximity_term_name, "effect_size"]

n_candidates_term_name <- "scale(n_candidates, scale = FALSE)"
if (!n_candidates_term_name %in% res$term) {
    n_candidates_term_name <- grep("n_candidates", res$term, value = TRUE)[1]
}
n_obj_est <- res[res$term == n_candidates_term_name, "Estimate"]
n_obj_se <- res[res$term == n_candidates_term_name, "Std. Error"]
n_obj_p_val <- res[res$term == n_candidates_term_name, "Pr(>|z|)"]
n_obj_eff_size <- res[res$term == n_candidates_term_name, "effect_size"]

actory_term_name <- "ActorY" # Assuming 'X' is the reference level for Actor factor
actory_est <- res[res$term == actory_term_name, "Estimate"]
actory_se <- res[res$term == actory_term_name, "Std. Error"]
actory_p_val <- res[res$term == actory_term_name, "Pr(>|z|)"]
actory_eff_size <- res[res$term == actory_term_name, "effect_size"]

# Prepare data for data frame
table_data <- list(
    c("StimulusID", "Variance", sprintf("%.7f", stim_id_var_val)),
    c("ParticipantID", "Variance", sprintf("%.7f", participant_id_var_val)),
    c("PromptID", "Variance", sprintf("%.7f", prompt_id_var_val)),
    c("Effect Size Denom", "", sprintf("%.7f", effect_size_denom_t)),

    c("Intercept", "b", sprintf("%.7f", intercept_est)),
    c("", "SE", sprintf("%.7f", intercept_se)),

    c("Angleleft", "b", sprintf("%.7f", angleleft_est)),
    c("", "SE", sprintf("%.7f", angleleft_se)),
    c("", "p", sprintf("%.7f", as.numeric(angleleft_p_val))),
    c("", "Effect size", sprintf("%.7f", angleleft_eff_size)),

    c("Angleright", "b", sprintf("%.7f", angleright_est)),
    c("", "SE", sprintf("%.7f", angleright_se)),
    c("", "p", sprintf("%.7f", as.numeric(angleright_p_val))),
    c("", "Effect size", sprintf("%.7f", angleright_eff_size)),

    c("Proximity", "b", sprintf("%.7f", proximity_est)),
    c("", "SE", sprintf("%.7f", proximity_se)),
    c("", "p", sprintf("%.7f", as.numeric(proximity_p_val))),
    c("", "Effect size", sprintf("%.7f", proximity_eff_size)),

    c("#Object", "b", sprintf("%.7f", n_obj_est)),
    c("", "SE", sprintf("%.7f", n_obj_se)),
    c("", "p", sprintf("%.7f", as.numeric(n_obj_p_val))),
    c("", "Effect size", sprintf("%.7f", n_obj_eff_size)),

    c("ActorY", "b", sprintf("%.7f", actory_est)),
    c("", "SE", sprintf("%.7f", actory_se)),
    c("", "p", sprintf("%.7f", as.numeric(actory_p_val))),
    c("", "Effect size", sprintf("%.7f", actory_eff_size))
)

# Convert to data frame
df_output <- do.call(rbind, lapply(table_data, function(x) data.frame(V1=x[1], V2=x[2], V3=x[3], stringsAsFactors=FALSE)))

# Print only the third column for easy copy-pasting to Excel
write.table(df_output[,3, drop=FALSE], file="", sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)

9.3504354
NA
NA
3.5553205
0.8427575
0.2180374
-0.1698470
0.2797391
0.5437433
-0.0477726
-0.1262396
0.2783831
0.6502072
-0.0355072
-0.4639305
0.1401113
0.0009291
-0.1304891
-0.5870535
0.1527254
0.0001211
-0.1651197
-0.9381818
0.2329477
0.0000564
-0.2638811
